導入套件

In [2]:
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd
%matplotlib inline

C:\ProgramData\Anaconda3\envs\tf23\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


載入資料_訓練集：2014 年 ~ 2018 年的 TSMC（共 1231 天）

In [4]:
dataset_train = pd.read_csv("台指數/台灣加權指數2014-2018.csv")
dataset_train = dataset_train.fillna(0) #缺值得都補 0，等等int轉換才不會有問題
dataset_train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-02,8618.599609,8632.809570,8587.540039,8612.540039,8612.507813,2537600.0
1,2014-01-03,8584.740234,8584.740234,8537.860352,8546.540039,8546.507813,2539700.0
2,2014-01-06,8553.000000,8568.240234,8488.639648,8500.009766,8499.977539,2640100.0
3,2014-01-07,8515.360352,8547.190430,8512.299805,8512.299805,8512.267578,2598900.0
4,2014-01-08,8548.610352,8587.080078,8548.610352,8556.009766,8555.977539,3206300.0


In [6]:
#將資料顯示為(資料長度,1) 等同reshape(-1,1)
training_set = dataset_train.iloc[: ,4:5].values
#training_set = dataset_train.iloc[: ,1:2].values
training_set

array([[8618.599609],
       [8584.740234],
       [8553.      ],
       ...,
       [9544.870117],
       [9555.009766],
       [9649.969727]])

In [39]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

#做 Normalization，將資料壓縮在 [0,1] 之間
sc = MinMaxScaler(feature_range = (0, 1))
print(sc)

training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled)

MinMaxScaler()
[[0.76534751]
 [0.75948247]
 [0.75534759]
 ...
 [0.84234402]
 [0.85679064]
 [0.86441969]]


In [47]:
X_train = []   #預測點的前 10 天的資料
y_train = []   #預測點
for i in range(10, len(training_set)):  # 1231 是訓練集總數 / 2014~2018總天數
    X_train.append(training_set_scaled[i-10:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)  # 轉成numpy array的格式，以利輸入 RNN

#因為現在 X_train 是 2-dimension，
#將它 reshape 成 3-dimension: [stock prices, timesteps, indicators]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print('第i天之前的股價')
print(X_train[0])
print('第i天股價')
print(y_train[0])


第i天之前的股價
[[0.76534751]
 [0.75948247]
 [0.75534759]
 [0.75643973]
 [0.76032399]
 [0.75665122]
 [0.75795485]
 [0.76122956]
 [0.75962462]
 [0.76445974]]
第i天股價
0.7653093307193404


In [50]:
# Import the Keras libraries and packages
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout,BatchNormalization

# Initialising the RNN
keras.backend.clear_session()
regressor = Sequential()

# 搭建 LSTM layer:

units: 神經元的數目 (setting 50)

第一層的 LSTM Layer 記得要設定input_shape參數

In [51]:
input_shape = (X_train.shape[1], 1)
print(input_shape)

(10, 1)


搭配使用dropout，這裡設為 0.2(Float between 0 and 1)

由於這邊的第四層 LSTM Layer 即將跟 Ouput Layer 做連接，因此注意這邊的 return_sequences 設為預設值 False （也就是不用寫上 return_sequences）

用了 2層 LSTM，之前4曾跑不出來是缺值忘了補0，轉int

In [52]:
# Adding the first LSTM layer and some Dropout regularisation,activation use relu functio
regressor.add(LSTM(units = 50,activation="relu", return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
#regressor.add(LSTM(units = 50,activation="relu", return_sequences = True))
#regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
#regressor.add(LSTM(units = 50, activation="relu",return_sequences = True))
#regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, activation="relu"))
regressor.add(Dropout(0.2))

# Adding the output layer setting 1
regressor.add(Dense(units = 1))

# Compiling & Fitting LSTM model
optimizer: 選擇 Adam \\\
loss: 使用 MSE \\\
learning rate(lr) :0.001 \\\
epochs :100 \\\
batch size:32

In [ ]:
# Compiling
optimizer = keras.optimizers.Adam(lr=0.001)
regressor.compile(optimizer = optimizer, loss = 'mean_squared_error')

# 進行訓練
history = regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
plt.title('train_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.plot( history.history["loss"])

Epoch 1/100
39/39 [==============================] - 1s 21ms/step - loss: 0.3294
Epoch 2/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0247
Epoch 3/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0219
Epoch 4/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0181
Epoch 5/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0181
Epoch 6/100
39/39 [==============================] - 1s 22ms/step - loss: 0.0164
Epoch 7/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0147
Epoch 8/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0162
Epoch 9/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0144
Epoch 10/100
39/39 [==============================] - 1s 22ms/step - loss: 0.0138
Epoch 11/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0131
Epoch 12/100
39/39 [==============================] - 1s 21ms/step - loss: 0.0129
Epoch 13/100
39/39 [=====

# 預測步驟

取測試集中 2019 年的股票資料（真實）

In [ ]:
dataset_test = pd.read_csv('台指數/台灣加權指數2019-2020.csv', usecols=[4], engine='python')
dataset_test.values

In [ ]:
dataset_test = pd.read_csv('台指數/台灣加權指數2019-2020.csv')
#real_stock_price = dataset_test.iloc[:, 4:5].values
real_stock_price = dataset_test.iloc[:, 1:2].values

取模型所預測的 2019 年股票資料（預測）

In [ ]:
dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) # Feature Scaling

X_test = []
for i in range(60, 80):  # timesteps一樣60； 80 = 先前的60天資料+2017年的20天資料
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))  # Reshape 成 3-dimension

start predict

In [ ]:
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)  # to get the original scale

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real TSEC Stock Price')  # 紅線表示真實股價
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted TSEC Stock Price')  # 藍線表示預測股價
plt.title('TSEC Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TSEC Stock Price')
plt.legend()
plt.show()